# Description of our juypter notebook usage
---

You should give the notebook a title and some explanations what is going on here!

## General Setup

In this section, the general setup of the notebook is done. This includes the import of the necessary modules and the definition of some global variables and the figure setup. Please don't change this without a good reason, and please let me know if you have a good reason to change it :)

### Module imports

In [1]:
# Standard imports
import numpy as np                                  # numerical computing
import pandas as pd                                 # dataframes

# For reading and writing files and directories
import os                                           # operating system 
import sys                                          # system specific parameters and functions    
from pathlib import Path                            # object-oriented filesystem paths

# Plotting library
import matplotlib as mpl                            # plotting
import matplotlib.pyplot as plt                     # plotting
import matplotlib.image as mpimg                    # plotting images
from matplotlib import style                        # plotting style

# Fitting and peak finding
from lmfit.models import LinearModel                # linear fit
from scipy.signal import savgol_filter, find_peaks  # for peak detection and smoothing

### Figure settings

In [2]:
# define which figure format to use, enable pdf and png here
save_as_pdf = 1
save_as_png = 1

# matpolotlib settings
mpl.rc('font', size=8, family="sans-serif", serif="Latin Modern") # set font size and family
mpl.rc('axes', titlesize=12)          # fontsize of the axes title
mpl.rc('axes', titlepad=6)            # pad between axis and title
mpl.rc('axes', labelsize=12)          # fontsize of the x and y labels
mpl.rc('xtick', labelsize=9)          # fontsize of the tick labels
mpl.rc('ytick', labelsize=9)          # fontsize of the tick labels
mpl.rc('legend', fontsize=8)          # legend fontsize
mpl.rc('figure', titlesize=10)        # fontsize of the figure title
mpl.rc('lines', markersize=2)         # default line markersize
mpl.rc('lines', marker="")            # default line marker
mpl.rc('grid', alpha=0.4)             # transparency of the grid
mpl.rc('grid', color="gray")          # color of the grid
mpl.rc('grid', linewidth=0.5)         # linewidth of the grid
mpl.rcParams['figure.constrained_layout.use'] = True
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["k", "r", "b", "g", "orange", "purple", "dimgray"]) # matplotlib colororder-default ändern
mpl.rcParams['lines.linewidth'] = 1.0  # adjust default line width

# adjust figure size according to the final document
# first, get width and height of tex document in inches

# For Toms scrartcl
# width = 6.69423

# For achemso Paper
width = 6.50128

# For Toms beamer
# width = 4.73726

# Finally, set the figure size
# height is same as width
fig_width = width * 0.5
fig_height = fig_width

### Useful Functions

These are some functions which you might find useful. Feel free to add more functions here.

In [3]:
def save_plot_as_png(fig: plt.figure, path: str) -> None:
    """save the plot as png.

    Parameters
    ----------
    fig : plt.figure
        figure object
    path : str
        path to save the figure
    """
    fig.savefig(path, format="png")

def save_plot_as_jpeg(fig: plt.figure, path: str) -> None:
    """save the plot as jpeg.

    Parameters
    ----------
    fig : plt.figure
        figure object
    path : str
        path to save the figure
    """
    fig.savefig(path, format="jpeg")
    
def save_plot_as_pdf(fig: plt.figure, path: str) -> None:
    """Save the plot as pdf.

    Parameters
    ----------
    fig : object
        figure object
    path : str
        path to save the figure
    """
    fig.savefig(path, format="pdf")

def getFileList(path: str, regex:str) -> list:
    """Get a list of files in a directory.

    Parameters
    ----------
    path : str
        path to the directory where the files are located
    regex : str
        regular expression to filter the files

    Returns
    -------
    list
        list of posix paths to the files
    """
    filelist = []
    for file in sorted(Path(path).rglob(regex)):
        filelist.append(file)

    if len(filelist) == 0:
        sys.exit("No '*data' files found.")

    return filelist

# some useful constants
eh2kcal = 627.503 # kcal/mol
ev2kcal = 23.0609 # kcal/mol
eh2kj = 2625.5 # kj/mol
ev2kj = 96.4869 # kj/mol
eh2ev = 27.2107 # electronvolt
ev2eh = 0.0367502 # hartree
eh2j = 43.60E-19 # Joule
kb = 1.38064852E-23 # Boltzmann constant
na = 6.02214076E23 # Avogadro constant

#### Evaluate RDFs and output results

In [4]:
def rdf_eval(df: pd.DataFrame, min_x_val_min: float, numbers_of_minima: int, numbers_of_maxima: int, height_min: float, height_max: float, distance_min: float,distance_max: float, prominence_min: float, prominence_max: float) -> tuple[list,list,list]:
    """Find minima and maxima of an RDF. This function is to be called by the function: rdf_table_to_latex.

    Parameters
    ----------
    df : pd.DataFrame
        dataframe containing the RDF data (r, g(r) and number integral (NI))
    min_x_val_min : float
        minimum x value to start the search for minima
    numbers_of_minima : int
        number of minima to find
    numbers_of_maxima : int
        number of maxima to find
    height_min : float
        minimum height of the minima to be found
    height_max : float
        minimum height of the maxima to be found
    distance_min : float
        minimum distance between minima
    distance_max : float
        minimum distance between maxima
    prominence_min : float
        minimum prominence of the minima to be found
    prominence_max : float
        minimum prominence of the maxima to be found

    Returns
    -------
    tuple: list, list, list
        positions of the maxima, positions of the minima and the number integral
        
    """

    
    # find maxima
    max = find_peaks(df["  g(r)"], height=height_max, threshold=None, distance=distance_max, prominence=prominence_max)
    
    maxima_pos = []
    max_height = []
    maxima_index = []
    for i in range(len(max[0])):
        max_height.append((max[1]['peak_heights'][i])*(-1))
        maxima_index.append(max[0][i])
        maxima_pos.append(df["# Distance / pm"][max[0][i]])

    # find minima
    min = find_peaks(df["  g(r)"]*(-1), height=height_min*(-1), threshold=None, distance=distance_min, prominence=prominence_min)
    
    minima_pos = []
    min_height = []
    minima_index = []
    for i in range(len(min[0])):
        if min[0][i] > min_x_val_min:
            min_height.append((min[1]['peak_heights'][i])*(-1))
            minima_index.append(min[0][i])
            minima_pos.append(df["# Distance / pm"][min[0][i]])

    # number integral at the first minimum
    NI_min = df["  Integral"][minima_index[0]]

    return (maxima_pos[0:numbers_of_maxima], minima_pos[0:numbers_of_minima], NI_min)

In [5]:
def rdf_to_latex_table(input_data: list, 
                       min_x_val_min: float = 200.0 , numbers_of_minima: int = 1, numbers_of_maxima: int = 1, height_min: float = 5.0, height_max: float = 1.0, distance_min: float = 50.0, distance_max: float = 100.0, prominence_min: float = 0.09, prominence_max: float = 0.05, 
                       sys_names: list = None, column_format: str = None, caption: str | tuple[str, str] = None, label: str = None, position: str = "htbp", save_to: str = None) -> None:
    """Evaluate RDFs and write them to a latex table.

    Parameters
    ----------
    input_data : list
        list of paths to the RDF files
    min_x_val_min : float, optional
        x value to start the search for minima with, by default 200.0  
    numbers_of_minima : int, optional
        number of minima to be found, by default 1
    numbers_of_maxima : int, optional
        number of maxima to be found, by default 1
    height_min : float, optional
        minimum height of the minima to be found, by default 1.0
    height_max : float, optional
        minimum height of the maxima to be found, by default 1.0
    distance_min : float, optional
        minimum distance between minima, by default 20.0
    distance_max : float, optional
        minimum distance between maxima, by default 20.0
    prominence_min : float, optional
        minimum prominence of the minima to be found, by default 0.05
    prominence_max : float, optional
        minimum prominence of the maxima to be found, by default 0.05
    sys_names : list, optional
        system names for the table, by default None
    column_format : str, optional
        column format of the latex table, by default None
    caption : str or tuple[str, str], optional
        caption of the latex table. if one, long caption of table, else (long cap, short cap) , by default None
    label : str, optional
        label of the latex table, by default None
    position : str, optional
        position of the latex table in the codument, by default "htbp"
    save_to : str, optional
        path to save the latex table, by default None
    """
    
    # initialize variables
    max_array = []
    min_array = []
    NI_min1 = []
    system = []

    # iterate over all files
    for i, data_path in enumerate(input_data):
        
        # load data
        df = pd.read_csv(data_path, sep=";", decimal='.')

        # smooth the graph
        df["  g(r)"] = savgol_filter(df["  g(r)"], 51, 3)

        # execute function find_the_peaks
        max, min, NI = rdf_eval(df, min_x_val_min, numbers_of_minima, numbers_of_maxima, height_min, height_max, distance_min, distance_max, prominence_min, prominence_max) 
        
        # append results
        max_array.append(float(str(max).replace("[","").replace("]","").replace(" ",",")))
        min_array.append(float(str(min).replace("[","").replace("]","").replace(" ",",")))
        NI_min1.append(float(NI))
        
        # name of the system
        # if there is a argument for the system name, use it
        if sys_names != None:
            system.append(sys_names[i])
        # else use the name of the file
        else:
            system.append(data_path.split("/")[-1])

    # create a data frame 
    table = pd.DataFrame({
                        r"\multicolumn{1}{c}{$r_\mathrm{max}$ / pm}": map(lambda x: "%.1f" % x, max_array), 
                        r"\multicolumn{1}{c}{$r_\mathrm{min}$ / pm}": map(lambda x: "%.1f" % x, min_array), 
                        r"\multicolumn{1}{c}{$NI(r_\mathrm{min})$}": map(lambda x: "%.2f" % x, NI_min1)}, 
                        index=system)
    
    # output the table
    display(table)
    # save the table
    table.style.to_latex(caption=caption, column_format=column_format, label=label, buf=save_to, hrules=True, position_float="centering", siunitx=False, position=position)

#### For contour plots

In [24]:
def generate_linear_color_map(maxval: float) -> mpl.colors.LinearSegmentedColormap:
    """Generate a linear color map.

    Parameters
    ----------
    maxval : float
        maximum value of the color map scale

    Returns
    -------
    mpl.colors.LinearSegmentedColormap
        linear color map
    """
    
    # set up a list of colors for the color scheme (no intensity = white)
    cont_colors = ["white", "lavender", "lightsteelblue", "cornflowerblue", "royalblue", "mediumblue", "blue", "darkcyan", "seagreen", "darkgreen", "green", "forestgreen", "limegreen", "lime", "lawngreen", "greenyellow", "yellow", "gold", "goldenrod", "orange", "darkorange", "coral", "salmon", "tomato", "orangered", "red", "crimson", "mediumvioletred", "deeppink", "magenta"]
    
    # create a list with values which assignes a value to each color
    color_values = maxval * np.linspace(0, 1, len(cont_colors))
    
    # the colorbar is normalized to the range of values in the data
    norm = plt.Normalize(min(color_values),max(color_values))
    
    # map(norm,color_values) creates a list of normalized values
    # zip() creates a list of tuples, where the first element of each tuple is a normalized value and the second element is a color
    # list() converts the zip object to a list
    tuples = list(zip(map(norm,color_values), cont_colors))

    # finially, create the linear color map from tuples
    cmap = mpl.colors.LinearSegmentedColormap.from_list("", tuples)
    
    # create a scalar mappable object for the colorbar
    cbar = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    
    # return the color map   
    return cmap, cbar

In [28]:
def plot_contour(file: str, **kwargs) -> None:
    
    """
    Function to generate a contour plot with colorbar 
    
    # Parameters
    
    file: string
        path to the data file which contains the z-values. 
        The data file has to be a `.csv` file with a semicolon as a delimiter. 
        It is designed to read in the `cdf*.gp.csv` files as generated by TRAVIS
    xvals, yvals: list / 1d-array, optional
        x- and y-values for the contour plot (Default: A list between 0 and the length of the data file in x-/y-direction)
    xlabel, ylabel: string, optional
        label for the x-axis and y-axis (default: None)
    xmin, xmax, ymin, ymax: number, optional
        limits for the x- and y-axes. (Default: The first and last values of the x-/y-values lists.)
    save_to: string, optional
        path to save the plot to in jpeg format. 
        No pdf because `matplotlib` has problems with rendering pdfs of contour plots (Default: None)
    """
    
    # read data and transpose them because TRAVIS exports them in the wrong order
    data = pd.read_csv(file, sep=";", header=None).transpose()
    
    # determine the maximum value of the data in order to set the colorbar limits
    maxval = data.max().max()                         
    
    # generate a linear color map using the function defined above
    cmap, cbar = generate_linear_color_map(maxval)
    
    # figure setup
    fig = plt.figure(figsize=(fig_width, fig_height))   
    gs = fig.add_gridspec(10,1)
    ax0 = fig.add_subplot(gs[0:9,0])            # contour plot
    ax1 = fig.add_subplot(gs[9,0])              # colorbar
    
    ##################################################################
    # check if the kwargs are given and set them accordingly
    # when doing a contour plot, the files output by TRAVIS contain the z-values, exclusively but no x- and y-values
    # the x and y values are arrays of the same length as the z-values and are generated here if not given as input
    
    # x and y data arrays
    # if they are not given as input, generate them as arrays of the same length as the z-values starting at 0 and ending at the length of the z-values
    if "xvals" in kwargs:
        xvals = kwargs["xvals"]
    else:
        xvals = np.linspace(0, data.shape[1], data.shape[1])
    if "yvals" in kwargs:
        yvals = kwargs["yvals"]
    else:
        yvals = np.linspace(0, data.shape[0], data.shape[0])
    
    # x and y axis limits
    # if they are not given, set them to the first and last values of the x and y data arrays
    if "xmin" in kwargs:
        xmin = kwargs["xmin"]
    else:
        xmin = xvals[0]
    if "xmax" in kwargs:
        xmax = kwargs["xmax"]
    else:
        xmax = xvals[-1]
    if "ymin" in kwargs:
        ymin = kwargs["ymin"]
    else:
        ymin = yvals[0]
    if "ymax" in kwargs:
        ymax = kwargs["ymax"]
    else:
        ymax = yvals[-1]
        
    # x and y axis labels
    if "xlabel" in kwargs:
        xlabel = kwargs["xlabel"]
    else:
        xlabel = ""
    if "ylabel" in kwargs:
        ylabel = kwargs["ylabel"]
    else:
        ylabel = ""
    
    ##################################################################
    
    ax0.set_xlabel(xlabel)
    ax0.set_ylabel(ylabel)
    ax0.set_xlim(xmin, xmax)
    ax0.set_ylim(ymin, ymax)
    
    # plot data
    ax0.contourf(xvals, yvals, data, cmap=cmap, levels=np.linspace(0, maxval, 100), extend="neither")
    ax0.grid(True)
    
    # colorbar
    ax1.set_axis_off()
    plt.colorbar(cbar,label=r"$\mathrm{Occurrence}$", location="bottom", fraction=0.5, pad=-0.7)
    
    # save figure
    if "save_to" in kwargs:
        plt.savefig(kwargs["save_to"], format="jpeg", dpi=1500)

---